In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# Build a Convolutional NN Using Estimators

In [3]:
from __future__ import absolute_import, division, \
    print_function, unicode_literals
import tensorflow as tf
import numpy as np

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
# Intro to Convolutional NNs
# Convolutional layers:
# Apply a specified number of convolution filters to the image.
# For each subregion, the layer performs a set of mathematical
# operations to produce a single value in the output feature map
# Then it applies a ReLU activation function to introduce
# nonlinearities.
# 
# Pooling layers:
# Downsample the image data extracted by the convolutional
# layers. This reduces the dimensionality of the feature map.
#
# Dense (fully connected) layers:
# Perform classification on the features extracted by the 
# convolutional layers and downsamped by the pooling layers.
# Every node in the layer is connected to every node in the 
# preceding layer.

In [6]:
# Building the CNN MNIST Classifier
# 1. Convolutional Layer #1 - Applies 32 5x5 filters with ReLU
# activation function.
# 2. Pooling Layer #1 - Performs max pooling with a 2x2 filter
# and stride of 2 so that pooled regions do not overlap
# 3. Convolutional Layer #2 - Applies 64 5x5 filters with ReLU 
# activation function.
# 4. Pooling Layer #2 - Performs max pooling with a 2x2 filter
# and stride of 2.
# 5. Dense Layer #1 - 1024 neurons, with dropout regularization
# rate of 0.4
# 6. Dense Layer #2 - 10 neurons, one for each digit target class
# 0-9

In [37]:
def cnn_model_fn(features, labels, mode):
    '''Model function for CNN'''
    # Input layer
    # Shape: [batch_size, img_height, img_width, channels]
    input_layer = tf.reshape(features["x"], [-1,28,28,1])
    # Convolutional layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer,\
                            filters=32,\
                            kernel_size=[5,5],\
                            padding="same",\
                            activation=tf.nn.relu)
    # Pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1,\
                                   pool_size=[2,2],\
                                   strides=2)
    # Convolutional layer #2
    conv2 = tf.layers.conv2d(inputs=pool1,\
                            filters=64,\
                            kernel_size=[5,5],\
                            padding="same",\
                            activation=tf.nn.relu)
    # Pooling layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2,\
                                   pool_size=[2,2],\
                                   strides=2)
    # Dense layer
    pool2_flat = tf.reshape(pool2, [-1,7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat, \
                           units=1024, \
                           activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, \
                training=mode == tf.estimator.ModeKeys.TRAIN)
    # Logits layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    predictions = {
        # Generate predictions (for PREDICT and eVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add 'softmax_tensor' to the graph. It is used for 
        # PREDICT and by the 'logging_hook.'
        "probabilities": tf.nn.softmax(logits, \
                                       name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,\
                        predictions=predictions)
    # Calculate loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(\
                labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(\
                        learning_rate=0.001)
        train_op = optimizer.minimize(\
                        loss=loss,\
                        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss,\
                            train_op=train_op)
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(\
            labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(\
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [38]:
# Load training and test data
((train_data, train_labels),(eval_data, eval_labels)) = \
    tf.keras.datasets.mnist.load_data()

In [39]:
train_data = train_data/np.float32(255)

In [40]:
train_labels = train_labels.astype(np.int32)

In [41]:
eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)

In [42]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(\
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7cbb0968dcc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [43]:
# Set up a logging hook
tensors_to_log = {"probabilities": "softmax_tensor"}

In [44]:
logging_hook = tf.train.LoggingTensorHook(\
    tensors=tensors_to_log, every_n_iter=50)

In [45]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(\
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [46]:
# Train one step and display the probabilities
mnist_classifier.train(input_fn=train_input_fn,\
                      steps=1, hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.08674837 0.10418858 0.10072877 0.11121342 0.1039355  0.10013232
  0.10096736 0.10192717 0.09527636 0.09488221]
 [0.10011754 0.08936473 0.11034227 0.11553523 0.09124382 0.09846262
  0.10671841 0.09069131 0.09878695 0.09873712]
 [0.09264695 0.10603054 0.10237494 0.10240807 0.10785181 0.10498852
  0.09048194 0.10029469 0.09591903 0.09700347]
 [0.08

INFO:tensorflow:loss = 2.3095558, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 2.3095558.


In [47]:
# Now, without logging each step, set steps=1000 to train the 
# model longer.
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.314947, step = 2
INFO:tensorflow:global_step/sec: 1.25624
INFO:tensorflow:loss = 2.2900202, step = 102 (79.613 sec)
INFO:tensorflow:global_step/sec: 1.1334
INFO:tensorflow:loss = 2.275901, step = 202 (88.227 sec)
INFO:tensorflow:global_step/sec: 1.226
INFO:tensorflow:loss = 2.270227, step = 302 (81.563 sec)
INFO:tensorflow:global_step/sec: 0.951835
INFO:tensorflow:loss = 2.2506065, step = 402 (105.063 sec)
INFO:ten

In [48]:
# Evaluate the model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [49]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-10-21T14:12:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-10-21-14:13:03
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.6717, global_step = 1001, loss = 2.0553105
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1001: /tmp/mnist_convnet_model/model.ckpt-1001


In [50]:
print(eval_results)

{'accuracy': 0.6717, 'loss': 2.0553105, 'global_step': 1001}
